<h1> Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [27]:
#import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

1.13.0-dev20181104


Read data created in the previous chapter.

In [28]:
# In CSV, label is the first column, after the features, followed by the key
df_full_ds = pd.read_csv('./inputs_n_targets.csv')

df_full_ds_shuffled = df_full_ds.sample(frac=1)
trainsize = int(df_full_ds_shuffled.shape[0] * 0.7)
validsize = int(df_full_ds_shuffled.shape[0] * 0.15)
testsize = int(df_full_ds_shuffled.shape[0] * 0.15)

LABEL_COLUMN = 0
LABELS_CAT = [0, 1, 2 ,3 ,4 ,5 ,6, 7, 8 ,9]
num_features = 200
FEATURES = range(1, num_features+1)

FEATURES_LABELS = ["acc_"+str(k) for k in FEATURES]

CSV_COLUMNS = ["number"] + FEATURES_LABELS
LABEL = "number"

df_train = df_full_ds_shuffled.iloc[:trainsize, :]
df_valid = df_full_ds_shuffled.iloc[trainsize:(trainsize+validsize), :]
df_test = df_full_ds_shuffled.iloc[validsize:(validsize+testsize), :]

def to_csv(df, filename):
  outdf = df.copy(deep=False)
  outdf.loc[:, 'key'] = np.arange(0, len(outdf)) # rownumber as key
  # reorder columns so that target is first column
  cols = outdf.columns.tolist()
  #cols.remove('fare_amount')
  #cols.insert(0, 'fare_amount')
  #print (cols)  # new order of columns
  outdf = outdf[cols]
  outdf.to_csv(filename, header=False, index_label=False, index=False)

def to_csv_with_headers(df, filename):
  outdf = df.copy(deep=False)
  #outdf.loc[:, 'key'] = np.arange(0, len(outdf)) # rownumber as key
  # reorder columns so that target is first column
  cols = outdf.columns.tolist()
  #cols.remove('fare_amount')
  #cols.insert(0, 'fare_amount')
  #print (cols)  # new order of columns
  outdf = outdf[cols]
  outdf.to_csv(filename, header=True, index_label=CSV_COLUMNS, index=False)

to_csv(df_train, 'acc-train.csv')
to_csv(df_valid, 'acc-valid.csv')
to_csv(df_test, 'acc-test.csv')

to_csv_with_headers(df_train, 'acc-train_labeled.csv')
to_csv_with_headers(df_valid, 'acc-valid_labeled.csv')
to_csv_with_headers(df_test, 'acc-test_labeled.csv')


print(FEATURES)
print(CSV_COLUMNS)
#df_full_ds.describe()
#print(df_full_ds)
#print(df_full_ds_shuffled)
#df_full_ds_shuffled.describe()
#df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
#df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)

range(1, 201)
['number', 'acc_1', 'acc_2', 'acc_3', 'acc_4', 'acc_5', 'acc_6', 'acc_7', 'acc_8', 'acc_9', 'acc_10', 'acc_11', 'acc_12', 'acc_13', 'acc_14', 'acc_15', 'acc_16', 'acc_17', 'acc_18', 'acc_19', 'acc_20', 'acc_21', 'acc_22', 'acc_23', 'acc_24', 'acc_25', 'acc_26', 'acc_27', 'acc_28', 'acc_29', 'acc_30', 'acc_31', 'acc_32', 'acc_33', 'acc_34', 'acc_35', 'acc_36', 'acc_37', 'acc_38', 'acc_39', 'acc_40', 'acc_41', 'acc_42', 'acc_43', 'acc_44', 'acc_45', 'acc_46', 'acc_47', 'acc_48', 'acc_49', 'acc_50', 'acc_51', 'acc_52', 'acc_53', 'acc_54', 'acc_55', 'acc_56', 'acc_57', 'acc_58', 'acc_59', 'acc_60', 'acc_61', 'acc_62', 'acc_63', 'acc_64', 'acc_65', 'acc_66', 'acc_67', 'acc_68', 'acc_69', 'acc_70', 'acc_71', 'acc_72', 'acc_73', 'acc_74', 'acc_75', 'acc_76', 'acc_77', 'acc_78', 'acc_79', 'acc_80', 'acc_81', 'acc_82', 'acc_83', 'acc_84', 'acc_85', 'acc_86', 'acc_87', 'acc_88', 'acc_89', 'acc_90', 'acc_91', 'acc_92', 'acc_93', 'acc_94', 'acc_95', 'acc_96', 'acc_97', 'acc_98', 'acc

In [19]:
df_train = pd.read_csv('acc-train.csv', header=None, names=CSV_COLUMNS)
df_valid = pd.read_csv('acc-valid.csv', header=None, names=CSV_COLUMNS)
df_test = pd.read_csv('acc-test.csv', header=None, names=CSV_COLUMNS)

df_train_labeled = pd.read_csv('acc-train_labeled.csv', header=None, names=CSV_COLUMNS, skiprows=1)
df_valid_labeled = pd.read_csv('acc-valid_labeled.csv', header=None, names=CSV_COLUMNS, skiprows=1)
df_test_labeled = pd.read_csv('acc-test_labeled.csv', header=None, names=CSV_COLUMNS, skiprows=1)
#df_test_labeled.describe()
print(df_train_labeled[LABEL])

0      3
1      3
2      4
3      2
4      1
5      8
6      8
7      3
8      3
9      8
10     9
11     2
12     7
13     1
14     4
15     6
16     5
17     1
18     0
19     8
20     0
21     9
22     9
23     9
24     0
25     0
26     3
27     4
28     2
29     6
      ..
320    0
321    0
322    1
323    6
324    7
325    5
326    6
327    7
328    0
329    4
330    7
331    0
332    4
333    9
334    3
335    6
336    7
337    3
338    7
339    8
340    6
341    0
342    4
343    8
344    1
345    3
346    3
347    9
348    7
349    8
Name: number, Length: 350, dtype: int64


<h2> Input function to read from Pandas Dataframe into tf.constant </h2>

In [29]:
def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df[FEATURES_LABELS],
    y = df[LABEL],
    batch_size = 200,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )





### Create feature columns for estimator

In [30]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES_LABELS]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [31]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'acc_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

#model = tf.estimator.LinearRegressor(
  #    feature_columns = make_feature_cols(), model_dir = OUTDIR)

#model.train(input_fn = make_input_fn(df_train, num_epochs = 10))

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [23]:
def print_rmse(model, name, df):
  metrics = model.evaluate(input_fn = make_input_fn(df, 1))
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
print_rmse(model, 'validation', df_valid)

NameError: name 'model' is not defined

This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [32]:
import itertools
# Read saved model and use it for prediction
#model = tf.estimator.LinearRegressor(
 #     feature_columns = make_feature_cols(), model_dir = OUTDIR)
#preds_iter = model.predict(input_fn = make_input_fn(df_valid, 1))
#print([pred['predictions'][0] for pred in list(itertools.islice(preds_iter, 5))])

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [33]:


tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNClassifier(hidden_units = [25],
      feature_columns = make_feature_cols(), model_dir = OUTDIR, n_classes=10)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd3754c10f0>, '_num_worker_replicas': 1, '_task_id': 0, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_global_id_in_cluster': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_evaluation_master': '', '_experimental_distribute': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_train_distribute': None, '_model_dir': 'acc_trained', '_is_chief': True, '_protocol': None, '_keep_checkpoint_every_n_hours': 10000, '_eval_distribute': None, '_device_fn': None, '_service': None, '_log_step_count_steps': 100, '_master': '', '_save_checkpoints_steps': None}


In [ ]:
model.train(input_fn = make_input_fn(df_train_labeled, num_epochs = 5000));
print_rmse(model, 'validation', df_valid_labeled)


We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

In [42]:
def json_serving_input_fn():
  """Build the serving inputs."""
  inputs = {}
  for feat in FEATURES_LABELS:
    inputs[feat] = tf.placeholder(shape=[None], dtype=np.int16)

  return tf.estimator.export.ServingInputReceiver(inputs, inputs)

model.export_savedmodel(export_dir_base='./acc_trained_exp', serving_input_receiver_fn=json_serving_input_fn, strip_default_attrs=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'classification' : Classification input must be a single string Tensor; got {'acc_117': <tf.Tensor 'Placeholder_116:0' shape=(?,) dtype=int16>, 'acc_179': <tf.Tensor 'Placeholder_178:0' shape=(?,) dtype=int16>, 'acc_186': <tf.Tensor 'Placeholder_185:0' shape=(?,) dtype=int16>, 

INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'acc_117': <tf.Tensor 'Placeholder_116:0' shape=(?,) dtype=int16>, 'acc_179': <tf.Tensor 'Placeholder_178:0' shape=(?,) dtype=int16>, 'acc_186': <tf.Tensor 'Placeholder_185:0' shape=(?,) dtype=int16>, 'acc_42': <tf.Tensor 'Placeholder_41:0' shape=(?,) dtype=int16>, 'acc_39': <tf.Tensor 'Placeholder_38:0' shape=(?,) dtype=int16>, 'acc_77': <tf.Tensor 'Placeholder_76:0' shape=(?,) dtype=int16>, 'acc_123': <tf.Tensor 'Placeholder_122:0' shape=(?,) dtype=int16>, 'acc_180': <tf.Tensor 'Placeholder_179:0' shape=(?,) dtype=int16>, 'acc_92': <tf.Tensor 'Placeholder_91:0' shape=(?,) dtype=int16>, 'acc_52': <tf.Tensor 'Placeholder_51:0' shape=(?,) dtype=int16>, 'acc_47': <tf.Tensor 'Placeholder_46:0' shape=(?,) dtype=int16>, 'acc_111': <tf.Tensor 'Placeholder_110:0' shape=(?,) dtype=int16>, 'acc_45': <tf.Tensor 'Placeholder_44:0' shape=(?,) dtype=int16>, 'acc_107': <tf.Tensor 'Placeholder_106:0' shape=(

INFO:tensorflow:Restoring parameters from acc_trained/model.ckpt-35000
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./acc_trained_exp/temp-b'1541390584'/saved_model.pb


b'./acc_trained_exp/1541390584'

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [113]:
def predict_input_fn(data_file):
    """ global CSV_COLUMNS
        CSV_COLUMNS = CSV_COLUMNS[:-1]
        df_data = pd.read_csv(
            tf.gfile.Open(data_file),
            names=CSV_COLUMNS,
            skipinitialspace=True,
            engine='python',
            skiprows=1
        )

        # remove NaN elements
        df_data = df_data.dropna(how='any', axis=0)
    """
    return tf.estimator.inputs.pandas_input_fn(
        x=df_test_labeled,
        num_epochs=1,
       shuffle=False
    )
results = model.predict(
        input_fn=predict_input_fn('acc-test_labeled.csv')
)
print(results)
#for result in results:
#    print 'result: {}'.format(result)
    
#print(df_test)

categories = [0, 1, 2 ,3 ,4 ,5 ,6, 7, 8 ,9]
expected = df_test_labeled[LABEL]
print(expected)

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(results, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    print(template.format(categories[class_id],
                          100 * probability, expec))


<generator object predict at 0x7f6a36ce9cd0>
0     4
1     1
2     6
3     9
4     7
5     2
6     6
7     0
8     3
9     9
10    9
11    7
12    5
13    5
14    2
15    1
16    5
17    6
18    4
19    3
20    3
21    2
22    2
23    9
24    9
25    1
26    0
27    8
28    5
29    4
     ..
45    6
46    0
47    5
48    7
49    8
50    4
51    4
52    6
53    6
54    5
55    4
56    5
57    2
58    5
59    5
60    8
61    8
62    3
63    8
64    5
65    5
66    7
67    7
68    2
69    2
70    7
71    7
72    3
73    7
74    6
Name: number, Length: 75, dtype: int64
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from acc_trained/model.ckpt-35000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "4" (100.0%), expected "4"

Prediction is "1" (100.0%), expected "1"

Prediction is "6" (100.0%), expected "6"

Prediction is "9" (100.0%), expected 

In [ ]:
import datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key,
  DAYOFWEEK(pickup_datetime)*1.0 AS dayofweek,
  HOUR(pickup_datetime)*1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 == {1}".format(base_query, phase)
  else:
    query = "{0} AND ABS(HASH(pickup_datetime)) % {1} == {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).to_dataframe()

In [ ]:
print_rmse(model, 'benchmark', df)

RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License